In [ ]:
# importing libraries necessary for dataframe creation 
import pandas as pd
from datetime import datetime
import requests
import json
import os
!pip install sodapy
from sodapy import Socrata

In [ ]:
#grabbing the data from the API

token="slJTBjNWDQSDAXdxtfz16w5iU"
dataset_identifier="rc75-m7u3"
client = Socrata("data.cityofnewyork.us", token)

metadata=client.get_metadata(dataset_identifier)
[x["name"] for x in metadata["columns"]]

query = """
select date_of_interest,
case_count,
probable_case_count,
death_count,
hospitalized_count,
bx_case_count,
bx_death_count,
bx_hospitalized_count,
bk_case_count,
bk_death_count,
bk_hospitalized_count,
mn_case_count,
mn_death_count,
mn_hospitalized_count,
qn_case_count,
qn_death_count,
qn_hospitalized_count,
si_case_count,
si_death_count,
si_hospitalized_count
"""

Covid_19 = client.get(dataset_identifier,query=query)

Covid_19_df = pd.DataFrame.from_records(Covid_19)

In [ ]:
#Cleaning the Data

Covid_19_dates=[]
for stuff in Covid_19_df["date_of_interest"]:
  stuff=stuff.replace('T00:00:00.000','')
  Covid_19_dates.append(stuff)


In [ ]:
Covid_19_df["date_of_interest"]=Covid_19_dates
Covid_19_df['date_of_interest']=pd.to_datetime(Covid_19_df['date_of_interest'],format = '%Y/%m/%d')
Covid_19_df[Covid_19_df.columns.difference(['date_of_interest'])]=Covid_19_df[Covid_19_df.columns.difference(['date_of_interest'])].apply(pd.to_numeric)

Covid_19_df["Year"] = pd.DatetimeIndex(Covid_19_df['date_of_interest']).year
Covid_19_df["Month"] = pd.DatetimeIndex(Covid_19_df['date_of_interest']).month
Covid_19_df["Day"] = pd.DatetimeIndex(Covid_19_df['date_of_interest']).day

In [ ]:
#installing Dash and libraries necessary for the Dashboard
!pip install dash
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output

     |████████████████████████████████| 9.6 MB 12.2 MB/s 
     |████████████████████████████████| 357 kB 65.7 MB/s 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  


In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([
                       
                       html.H1("Web Application Dashboard with DAsh", style ={'text-align': 'center'}),

                       dcc.Dropdown(id = "Select_Year",
                                    options = [
                                               {'label':'2020', 'value':2020},
                                               {'label':'2021', 'value':2021},
                                               {'label':'2022', 'value':2022}],
                                    multi = False,
                                    value = 2020,
                                    style = {'width': "40%"}
                                    ),
                       html.Div(id="output_container" , children = []),
                       html.Br(),
                       dcc.Graph(id = 'Covid_19_US_map',figure = {})
])

